In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [ ]:
df = pd.read_csv('../asset/inter_240113_title_none_piv.csv')
df = df[df['local_time'] >= '2024-01-15 00:00:00+00:00']
df

In [ ]:
with open('../asset/product_info_with_title.pickle', 'rb') as fr:
    product_data = pickle.load(fr)

product_data

## hashed_ip 별 title 묶기

In [ ]:
data_drop_local_time = df.drop(columns='local_time', axis=0)
grouped = data_drop_local_time.groupby('hashed_ip')

In [ ]:
df_grouped = pd.DataFrame(columns=['hashed_ip', 'products', 'product_titles', 'last_product', 'last_product_title'])

for i in tqdm(grouped):
    if len(i[1]) > 1 :
        product_id_list = []
        product_title_list = []
        
        # 상품명 순서에 따라 유사도에 변화가 있어, 최근에 본 상품일수록 앞쪽에 등장하도록 하였습니다.
        for j in i[1].index.sort_values(ascending=False)[1:]:
            if i[1].loc[j, 'products'] in product_id_list:
                pass
            else:
                product_id_list.append(i[1].loc[j, 'products'])
                product_title_list.append(i[1].loc[j, 'title'])

        df_grouped.loc[len(df_grouped)] = [i[0], product_id_list, product_title_list, list(set(i[1]['products'][-1:])), list(set(i[1]['title'][-1:]))]

df_grouped
        

## fastText

In [ ]:
from gensim.models.fasttext import FastText
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
product_document = []
for i in tqdm(range(len(product_data))):
    product_document.append([product_data.loc[i, 'title']])

In [ ]:
# parameter
window = 5
min_count = 1
epochs = 50

In [ ]:
# FastText_Skip-Gram with Negative Sampling
model_sgns = FastText(product_document, vector_size =100, window=window, min_count=min_count, workers=10, hs=1, sg=1)
model_sgns.train(product_document,total_examples=len(product_document),epochs=epochs)

---
---

In [ ]:
def hit_at_k(model, topn):
    hit = 0
    for i in tqdm(range(len(df_grouped))):
        # i 번째 유저와의 유사도 top10 item title
        target = ' '.join(df_grouped.loc[i, 'product_titles'])
        similar_product = [prod for prod, sim in model.wv.most_similar(positive=target, topn=topn)]
        # 10개에 이 유저가 마지막으로 본 아이템이 속해 있는가? 있으면 hit10에 +1
        test = set(similar_product) & set(df_grouped.loc[i, 'last_product_title'])
        if len(test):
            hit +=1

    print(f"hit@{topn} : ", hit/len(df_grouped))

In [ ]:
hit_at_k(model_sgns, 100)

---
---

In [ ]:
def testing(model, target):
    print(target)
    print('-'*20)
    
    for prod, sim in model.wv.most_similar(positive=target, topn=10):
        print(prod, '|', sim)

In [ ]:
# product title을 document로 => 키워드 알림용 => list 형태인 product title을 join하여 input으로
testing(model_sgns, ' '.join(df_grouped.loc[555, 'product_titles']))

In [ ]:
# product title을 document로(자기 자신 결과 안 나오도록) => 추천 배너용 => list 형태인 product title을 그대로 input으로
testing(model_sgns, df_grouped.loc[0, 'product_titles'])

In [ ]:
# product title을 document로
testing(model_sgns, ['애플 아이패드'])